In [7]:
import cv2
import pyautogui
import numpy as np
from keras.models import load_model
import keyboard
import cv2 as cv 
import numpy as np
import matplotlib.pyplot as plt
import mss.tools
import time
import uuid
import keyboard
import pyautogui
import time
import mss.tools
import uuid
import keyboard
from pynput import mouse

In [5]:
def screenshot():
    with mss.mss() as sct:
        time.sleep(1)
        monitor_info = sct.monitors[0]
        top_margin = (monitor_info["height"] // 3) + 50
        monitor = {"top": monitor_info["top"] + top_margin, "left": monitor_info["left"] + 55, "width": monitor_info["width"] - 120, "height": (monitor_info["height"] * 2 // 3)-105}
        output = f"testimages/{str(uuid.uuid4())}.png".format(**monitor)
        sct_img = sct.grab(monitor)
        mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
    return output

In [8]:
model = load_model('model_v2_iris.h5')

In [24]:
img_orig = cv.imread("DLIngrid\q/3bd9faf6-2375-4d13-b484-b468c8ae9f02.png")
img_rgb = cv.cvtColor(img_orig, cv.COLOR_BGR2RGB)
reshaped_image = np.expand_dims(img_rgb, axis=0)

In [42]:
def preprocess_image(input_image):
    # Resize the image to match the input size of your model
    original = cv.imread(input_image)
    colored = cv.cvtColor(original, cv.COLOR_BGR2RGB)
    resize = cv2.resize(colored, (224, 224))
    # Normalize pixel values to be in the range [0, 1]
    normalized = resize / 255.0
    # Expand dimensions to match the input shape expected by the model
    dimension = np.expand_dims(normalized, axis=0)
    y = preprocess_input(dimension)
    return y

In [57]:
prediction = model.predict(preprocess_image("./madness/train/q/05a1776e-86c6-4ffe-a619-48e69aaf29bb.png"))

1/1 [==============================] - 0s 113ms/step


In [58]:
action_index = np.argmax(prediction)
confidence = prediction[0][action_index]

In [59]:
action_index = np.argmax(prediction)
confidence = prediction[0][action_index]
print(action_index)
print(confidence)

0
0.66032183


In [60]:
prediction[0]

array([0.66032183, 0.25085917, 0.08881905], dtype=float32)

In [ ]:
time.sleep(5)
while not keyboard.is_pressed("space"):
    #test = 
    image = screenshot()


    

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.optimizers as optimizers
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten
from keras.layers import Reshape

In [39]:
import splitfolders
input_folder = './dlcombineddata'
output_folder = './combinedsplit'

#split the images into 80% training and 20% testing sets
splitfolders.ratio(input_folder, output_folder, seed=40, ratio=(0.8, 0.2))

Copying files: 1837 files [00:04, 383.40 files/s]


In [2]:
model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(224, 224, 1))

In [3]:
for layer in model.layers:
    layer.trainable = False

In [4]:
x = model.output


#x = Reshape((1, 1, -1))(x)
#x = GlobalAveragePooling2D()(x)

x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)  # Example convolutional layer
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
classes = Dense(3, activation='softmax')(x) 


model = Model(inputs=model.input, outputs=classes)
optimizer = optimizers.Adam(learning_rate=0.01)


In [5]:
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [7]:
datagen = ImageDataGenerator(rescale=1./255,
                                #rotation_range=5,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                #zoom_range=0.1,
                                #shear_range=0.2,
                                channel_shift_range=0.2,
                                brightness_range=(0.8, 1.2),
                                #contrast_range=(0.9, 1.1),
                                #saturation_range=(0.8, 1.2),
                                
                                #horizontal_flip=True
                                )

In [8]:
train_data = datagen.flow_from_directory("./combinedsplit/train", target_size=(270, 1800), batch_size=23, class_mode="categorical")
val_data = datagen.flow_from_directory("./combinedsplit/val", target_size=(270, 1800), batch_size=23, class_mode="categorical")

Found 1468 images belonging to 3 classes.
Found 369 images belonging to 3 classes.


In [8]:
print(train_data.class_indices)

{'d': 0, 'forward': 1, 'q': 2}


In [9]:
model.fit(train_data, epochs=5, validation_data=val_data)

Epoch 1/5


 6/64 [=>............................] - ETA: 5:33 - loss: 13.5970 - accuracy: 0.3043

In [34]:
optimizer = optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [33]:
for layer in model.layers:
    layer.trainable = True

In [32]:
resnet_model = model.layers[0] # assuming the ResNet50 model is the first layer of your model
for layer in resnet_model.layers[:-10]:
    layer.trainable = False
for layer in resnet_model.layers[-10:]:
    layer.trainable = True

AttributeError: 'InputLayer' object has no attribute 'layers'

In [35]:
model.fit(train_data, epochs=5, validation_data=val_data)

Epoch 1/5
13/13 [==============================] - 158s 11s/step - loss: 0.3631 - accuracy: 0.8937 - val_loss: 0.2995 - val_accuracy: 0.9133
Epoch 2/5
13/13 [==============================] - 144s 11s/step - loss: 0.2268 - accuracy: 0.9210 - val_loss: 0.3180 - val_accuracy: 0.9079
Epoch 3/5
13/13 [==============================] - 141s 11s/step - loss: 0.2109 - accuracy: 0.9319 - val_loss: 0.3228 - val_accuracy: 0.9133
Epoch 4/5
13/13 [==============================] - 144s 11s/step - loss: 0.1831 - accuracy: 0.9305 - val_loss: 0.3367 - val_accuracy: 0.9079
Epoch 5/5
13/13 [==============================] - 146s 11s/step - loss: 0.1677 - accuracy: 0.9394 - val_loss: 0.3864 - val_accuracy: 0.9079


In [88]:
model.save('goodmaybe.keras')

In [90]:
model.save('goodmaybe.h5')

c:\Users\ingri\anaconda3\envs\mss\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [177]:
modelsaved = load_model('goodmaybe.keras')

In [20]:
img = cv.imread("croppeddataingrid\d/0c89b0ea-6314-415f-a6fb-b9772f845f7b.png")
img_rgb = cv.cvtColor(img_orig, cv.COLOR_BGR2RGB)
img = cv2.resize(img_rgb, (224, 224))
img_resized = img.astype('float32') / 255.0 
reshaped_image = np.expand_dims(img, axis=0)

In [19]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.utils import load_img

In [36]:
def preprocess_image(input_image):
    # Resize the image to match the input size of your model
    original = cv.imread(input_image)
    #colored = cv.cvtColor(original, cv.COLOR_BGR2RGB)
    #resize = cv2.resize(colored, (270, 1800))
    # Normalize pixel values to be in the range [0, 1]
    normalized = original / 255.0
    # Expand dimensions to match the input shape expected by the model
    dimension = np.expand_dims(normalized, axis=0)
    y = preprocess_input(dimension)
    return y

In [14]:
class_names = open("./converted_keras/labels.txt", "r").readlines()

In [2]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL

In [15]:
def predict_direction():
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    path = "combinedsplit/train\q/0aa79513-668b-4f74-8ac1-66400df250a2.png"
    image = Image.open(path).convert("RGB")
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    data[0] = normalized_image_array
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    class_name = class_name[:1]
    confidence_score = prediction[0]
    print(confidence_score)
    return class_name

In [41]:
hmm = model.predict(preprocess_image("combinedsplit/train/forward/1d49b846-a162-4bd4-9986-22042d25e40a.png"))

1/1 [==============================] - 0s 392ms/step


In [42]:
action_index = np.argmax(hmm)
confidence = hmm[0][action_index]
print(action_index)
print(confidence)

1
0.8329307


In [34]:
hmm[0]

array([0.66031635, 0.2508504 , 0.08883328], dtype=float32)

In [16]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import resnet_v2

# The good shit

In [4]:
why = load_model('colab.h5')

In [5]:
###################################################THIS SHIT IS THE GGOD SHIT
img = image.load_img("combinedsplit/val\q/5cbaa01a-8d81-417b-860f-141024727613.png", target_size=(224, 224))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Expand the dimensions to match the shape that the model expects
x = np.expand_dims(x, axis=0)

# Preprocess the input
x = resnet_v2.preprocess_input(x)

In [6]:
bruh = why.predict(x)

1/1 [==============================] - 1s 1s/step


In [7]:
action_index = np.argmax(bruh)
confidence = bruh[0][action_index]
print(action_index)
print(confidence)

2
0.99999976


In [1]:
import os
import numpy as np
from tensorflow.keras.applications import resnet_v2
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [44]:
directory = 'combinedsplit/val'

# Lists to store the actual and predicted classes
y_true = []
y_pred = []

le = LabelEncoder()

# Iterate over every directory in the directory
for subdir in os.listdir(directory):
    subdir_path = os.path.join(directory, subdir)
    if os.path.isdir(subdir_path):
        # Iterate over every image in the subdirectory
        for filename in os.listdir(subdir_path):
            if filename.endswith(".png"): 
                # Load the image
                img_path = os.path.join(subdir_path, filename)
                img = image.load_img(img_path, target_size=(224, 224))

                # Convert the image to a numpy array
                x = image.img_to_array(img)

                # Expand the dimensions to match the shape that the model expects
                x = np.expand_dims(x, axis=0)

                # Preprocess the input
                x = resnet_v2.preprocess_input(x)

                # Make a prediction
                predictions = why.predict(x)

                # Get the index of the class with the highest probability
                predicted_class = np.argmax(predictions)

                # Add the actual and predicted classes to the lists
                # The actual class is the name of the subdirectory
                y_true.append(subdir)
                y_pred.append(predicted_class)

                continue
            else:
                continue

y_true = le.fit_transform(y_true)
# Create a confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print(cm)

1/1 [==============================] - 0s 62ms/step
[[109   5   1]
 [ 27  96   4]
 [ 19   7 101]]


In [45]:
# Print the classes
for i, class_name in enumerate(le.classes_):
    print(f"Encoded label {i} corresponds to original label '{class_name}'")

Encoded label 0 corresponds to original label 'd'
Encoded label 1 corresponds to original label 'forward'
Encoded label 2 corresponds to original label 'q'
